In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import PIL.Image as Image
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import os
import progressbar
from time import sleep

dim = (224, 224)
data_path = "./dataset/Images/"
#labels_path = "./dataset/labels.txt"

In [2]:
def load_image(name) :
    img = Image.open(name).resize(dim)
    img = np.array(img)/255.0
    return img

In [ ]:
bar = progressbar.ProgressBar(maxval=20580, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
images = []
labels = []
i=1
for root, dirs, files in os.walk(data_path, topdown=False):
    for name in files:
        images=np.append(images,load_image(os.path.join(root, name)))
        labels=np.append(labels, root[17:])
        bar.update(i)
        i=i+1
bar.finish()

In [21]:
print(len(images))
print(len(labels))
np.savetxt('./dataset/Images/nparrayimages.txt',images)
np.savetxt('./dataset/Images/nparraylabels.txt',labels)

0

In [ ]:
npimages=np.loadtxt('./dataset/Images/nparrayimages.txt')
nplabels=np.loadtxt('./dataset/Images/nparraylabels.txt')
print(npimages == images)
print(nplabels == labels)

In [8]:
kf = KFold(n_splits=5, random_state=None, shuffle=True)
kf.get_n_splits(npimages)
X = npimages
y = nplabels
mobilenet = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                            input_shape=(224,224,3))
mobilenet.trainable = False
model = tf.keras.Sequential([mobilenet, layers.Dense(120, activation='softmax')])
model.summary()

644


In [5]:
print("starting cross val for 5 folds")
i = 1
for train_index, test_index in kf.split(X):
    trainData = X[train_index]
    testData = X[test_index]
    trainLabels = y[train_index]
    testLabels = y[test_index]
    print("\n********* K Fold Cross Val #",i)
    
    model = tf.keras.Sequential([mobilenet, layers.Dense(120, activation='softmax')])
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    num_batches = np.ceil(training_images.samples/training_images.batch_size)
    _=model.fit(trainData,trainLabels,batch_size=64,epochs=10,validation_data=(testData,testLabels))
    i=i+1

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()